In [ ]:
import pm4py
import os
import pandas as pd

# Q1. Inductive Miner

> In this question you should discover a model for the given event log with a special focus on the Inductive Miner implemented in PM4Py.

## Data loading and exploration

In [ ]:
data_path = "log.csv"
figures_dir = "./figures/"

assert os.path.exists(data_path)

# load csv and rename patieent and activity column
df_log = pd.read_csv(data_path)

# convert timestamp columns to datetime friendly format
df_log['Timestamp'] = pd.to_datetime(df_log['Timestamp'])
df_log['start_timestamp'] = pd.to_datetime(df_log['start_timestamp'])

# rename certain columns to ensure recognition by pm4py
df_log = df_log.rename(columns={"Age": "case:Age", "Insurance": "case:Insurance", "PatientName": "case:PatientName"})

df_log.head()

In [ ]:
#from pandas_profiling import ProfileReport

#log_profile = ProfileReport(df_log, title="Raw Log Profile")

#log_profile.to_notebook_iframe()

We first verify if the `Patient` has repeated traces, i.e., if it is used again in case the patient returns to the institution. We can test this through the `PatientName` column.

In [ ]:
# count unique patient identification number for each patient name
patients_n_ids = df_log.groupby('case:PatientName')['Patient'].nunique()

(patients_n_ids > 1).any()

So no patient has more than one ID, which means we can use the `Patient` column as an identifier of the traces.

Let's take a look at some traces:

In [ ]:
import numpy as np

# group activities into traces
traces = df_log.groupby('Patient')['Activity'].apply(np.array)

for trace in traces.sample(5).values:
    print(' -> '.join(trace))
    print()

Now we transform the dataset into an event log, proper for analysis.

In [ ]:
from pm4py.objects.conversion.log import factory as log_converter
from pm4py.util import constants

# map dataset columns to PM4Py keys
param_keys={constants.PARAMETER_CONSTANT_CASEID_KEY: 'Patient',
            constants.PARAMETER_CONSTANT_RESOURCE_KEY: 'Resource', 
            constants.PARAMETER_CONSTANT_ACTIVITY_KEY: 'Activity',
            constants.PARAMETER_CONSTANT_TIMESTAMP_KEY: 'Timestamp',
            constants.PARAMETER_CONSTANT_START_TIMESTAMP_KEY: 'start_timestamp'}

event_log = log_converter.apply(df_log, parameters=param_keys)

event_log

## a)

> Apply the Inductive Miner implemented in PM4Py to the given event log and describe the process. Furthermore, give and reason about the fitness and precision results, respectively. On a high level, describe the potential problems of the model and reason how they were caused by the algorithm and the log.

We generate both the petri net and the process tree, the two possible outcomes of the implementation of the inductive miner.

In [ ]:
from pm4py.algo.discovery.inductive import factory as inductive_miner
from pm4py.visualization.petrinet import factory as pn_visualizer

net, initial_marking, final_marking = inductive_miner.apply(event_log, parameters=param_keys)

net_graph = pn_visualizer.apply(net, initial_marking=initial_marking, final_marking=final_marking,
                                log=event_log, parameters=param_keys)

# fix places size in the graph
import numpy as np
body = np.array(net_graph.body)
body[body ==  '\tnode [fixedsize=true shape=circle width=0.75]'] = '\tnode [fixedsize=true shape=circle width=1]'
net_graph.body = body

net_graph.render(os.path.join(figures_dir, 'q1_a_petrinet'), format='pdf', view=True)

In [ ]:
from pm4py.visualization.process_tree import factory as pt_visualizer

tree = inductive_miner.apply_tree(event_log, parameters=param_keys)

tree_graph = pt_visualizer.apply(tree)

# fix operations size in the graph
tree_graph.body = list(map(lambda r:r.replace('width=0.6', 'width=1'),
                           tree_graph.body))

tree_graph.render(os.path.join(figures_dir, 'q1_a_tree'),
                 format='pdf',
                 view=True)

## b)

> From the process owner we know that patients are called in order to control the quarantine and that there are two potential quarantine phases, i.e., before and after a positive test. Implement a function that resolves the duplicate activity Control Call by context sensitive renaming. Discuss the impact on the discovered model.

Let's assume that the `Test III` is what distinguishes the two `Control Call` possibilities. We'll rename the one after the positive test as `Control Call (+)`.

In [ ]:
new_cc_name = 'Control Call (+)'
split_on_activity = 'Test III'

# get `Test III` moment
df_log_test = df_log[df_log['Activity'] == split_on_activity]
split_timestamp = df_log_test.groupby('Patient')['Timestamp'].first()

# map timestamp to whole patient trace
df_renaming = df_log.copy()
df_renaming[split_on_activity + ' Timestamp'] = df_log['Patient'].map(split_timestamp)

# renames `Control Call` activities that happen after `Test III`
new_cc = df_renaming['Activity'].str.replace('Control Call', new_cc_name)
df_renaming['Activity'] = new_cc.where(
    df_renaming['Timestamp'] > df_renaming[split_on_activity + ' Timestamp'],
    df_renaming['Activity']
)
df_renaming['Activity'].value_counts()

Check if it worked.

In [ ]:
df_renaming['Activity'].value_counts().drop(['Control Call', new_cc_name]) - \
    df_log['Activity'].value_counts().drop('Control Call')

In [ ]:
renamed_traces = df_renaming.groupby('Patient')['Activity'].apply(np.array)

renamed_traces.sample(10).values

No problems detected.

Generate the tree.

In [ ]:
# generate a new event log based on the modified dataset
renamed_event_log = log_converter.apply(df_renaming, parameters=param_keys)

#save the renamed event log
from pm4py.objects.log.exporter.xes import factory as xes_exporter
log_path = "processed_log.xes"
xes_exporter.apply(renamed_event_log, log_path)

tree = inductive_miner.apply_tree(renamed_event_log, parameters=param_keys)

tree_graph = pt_visualizer.apply(tree)

# fix operations size
tree_graph.body = list(map(lambda r:r.replace('width=0.6', 'width=1'),
                           tree_graph.body))

tree_graph.render(os.path.join(figures_dir, 'q1_b_tree'),
                 format='pdf',
                 view=True)

## c)

> The log contains a considerable amount of noise induced by errors during the event logging. Apply the IM to a DFG filtered for noise. Describe your results and explain why the IM mines a different model. Which type of noise is prominent in the log?

First we generate the DFG with the log after context-sensitive renaming.

In [ ]:
from pm4py.algo.discovery.dfg import factory as dfg_discovery
from pm4py.visualization.dfg import factory as dfg_visualization

dfg = dfg_discovery.apply(renamed_event_log, parameters=param_keys)

dfg_viz = dfg_visualization.apply(dfg, log=renamed_event_log, parameters=param_keys)

dfg_viz.render(os.path.join(figures_dir, 'q1_c_dfg_renaming'),
               format='pdf',
               view=True)

Then we filter the graph.

In [ ]:
from pm4py.objects.dfg.filtering import dfg_filtering

filt_dfg = dfg_filtering.apply(dfg, parameters=param_keys)

filt_dfg_viz = dfg_visualization.apply(filt_dfg, log=renamed_event_log, parameters=param_keys)

filt_dfg_viz.render(os.path.join(figures_dir, 'q1_c_dfg_filtered'),
                    format='pdf',
                    view=True)

We can already notice the improvement through the DFG, which is much more readable. There are way less edges and the flow seems much more linear. Even though, the structure about the treatments A1, A2 and A3 is still unsettled.

Finally, we apply the IM algorithm again.

In [ ]:
filt_tree = inductive_miner.apply_tree_dfg(filt_dfg, parameters=param_keys)

tree_graph = pt_visualizer.apply(filt_tree)

# fix operations size
tree_graph.body = list(map(lambda r:r.replace('width=0.6', 'width=1'),
                           tree_graph.body))

tree_graph.render(os.path.join(figures_dir, 'q1_c_filt_tree'),
                  format='pdf',
                  view=True)

The new model generated is much closer to the expected from the superficial analysis of the traces. The noise removed comes mostly from skipped activities, as we can notice that the edges in the DFG reduced and, therefore the amount of silent transitions as well. Still, there are plenty of them, related to the `Treatment A*` activities.

Let's check the traces in which these activities are present.

In [ ]:
treatment_A_traces = renamed_traces[renamed_traces.apply(lambda t: any('Treatment A' in a for a in t))]

treatment_A_traces.shape[0]

In [ ]:
treatment_A_traces.sample(10).values

We can notice how the rarity of traces with `Treatment A*` activities impacted in them being filtered, as the transitions from `Decide Treatment` to the `Treatment A*` were filtered out. But as the `Treatment A*` happen many times in a single trace, the structure was roughly kept.

## d)

> Investigate the DFG of the log after applying the preceding steps. Which activities might be filtered out in order to obtain an improved model that explains most of the process more precisely? Why might this yield better results when applying the IM? Implement a filter and apply the IM to the filtered log.

Perhaps the resulting model can be improved by tuning the noise threshold or by removing the `* Treatment A*` activities completely.

#### Total `* Treatment A*` removal

As the traces in which the A* treatments were applied account for only 123 traces (8,2%), we can say that removing them totally we will still represent the majority of the process.

In [ ]:
df_no_A = df_renaming[~df_renaming['Patient'].isin(treatment_A_traces.index)]

df_no_A

In [ ]:
log_no_A = log_converter.apply(df_no_A, parameters=param_keys)

dfg_no_A = dfg_discovery.apply(log_no_A, parameters=param_keys)

dfg_no_A = dfg_filtering.apply(dfg_no_A, parameters=param_keys)

dfg_viz = dfg_visualization.apply(dfg_no_A, log=log_no_A, parameters=param_keys)

dfg_viz.render(os.path.join(figures_dir, 'q1_d_no_A'),
               format='pdf',
               view=True)

The DFG now looks very concise.

In [ ]:
tree_no_A = inductive_miner.apply_tree_dfg(dfg_no_A, parameters=param_keys)

tree_graph = pt_visualizer.apply(tree_no_A)

# fix operations size
tree_graph.body = list(map(lambda r:r.replace('width=0.6', 'width=1'),
                           tree_graph.body))

tree_graph.render(os.path.join(figures_dir, 'q1_d_tree_no_A'),
                  format='pdf',
                  view=True)

We can see that the PT is also much closer to the majority of the traces, representing well the process. Still, it keeps the silent activities.

#### Partial `* Treatment A*` removal

We can also remove only the activities related to these treatments, keeping the traces. This ensures that we do not loose too much data.

In [ ]:
df_silent_A = df_renaming[df_renaming['Activity'].str.find('Treatment A') == -1]
df_silent_A

In [ ]:
log_silent_A = log_converter.apply(df_silent_A, parameters=param_keys)

dfg_silent_A = dfg_discovery.apply(log_silent_A, parameters=param_keys)

dfg_silent_A = dfg_filtering.apply(dfg_silent_A, parameters=param_keys)

dfg_viz = dfg_visualization.apply(dfg_silent_A, log=log_silent_A, parameters=param_keys)

dfg_viz.render(os.path.join(figures_dir, 'q1_d_silent_A'),
               format='pdf',
               view=True)

We notice this way the `Prescripe Special Medication` activity is kept.

In [ ]:
tree_silent_A = inductive_miner.apply_tree_dfg(dfg_silent_A, parameters=param_keys)

tree_graph = pt_visualizer.apply(tree_silent_A)

# fix operations size
tree_graph.body = list(map(lambda r:r.replace('width=0.6', 'width=1'),
                           tree_graph.body))

tree_graph.render(os.path.join(figures_dir, 'q1_d_tree_silent_A'),
                  format='pdf',
                  view=True)

The biggest difference in the Process Tree is that the `Discharge Test` activity was dragged away from the exclusive choice operator with the other 3 activities to a different exclusive choice with `Prescripe Special Medication`.

#### Noise threshold tuning

For the reasons stated in c), the noise threshold will be tuned.

In [ ]:
param_keys['noiseThreshold'] = 0.07

filt_dfg = dfg_filtering.apply(dfg, parameters=param_keys)

filt_dfg_viz = dfg_visualization.apply(filt_dfg, log=renamed_event_log, parameters=param_keys)

filt_dfg_viz.render(os.path.join(figures_dir, 'q1_d_dfg_noise_threshold_tuning'),
                    format='pdf',
                    view=True)

We can already notice the improvement in the DFG, which is much more readable. There are way less edges and the flow seems much more linear. Even though, the structure about the treatments A1, A2 and A3 is still unsettled.

In [ ]:
filt_tree = inductive_miner.apply_tree_dfg(filt_dfg, parameters=param_keys)

tree_graph = pt_visualizer.apply(filt_tree)

# fix operations size
tree_graph.body = list(map(lambda r:r.replace('width=0.6', 'width=1'),
                           tree_graph.body))

tree_graph.render(os.path.join(figures_dir, 'q1_d_noise_threshold_tuning'),
                  format='pdf',
                  view=True)

Even though the final model is not so readable and can still accept some unreasonable mixes between applying one treatment and checking for another, one can say that it is more complete as to the amount of structures that it presents.

In [ ]:
from pm4py.objects.conversion.process_tree import factory as pt_converter
from pm4py.objects.petri.exporter import factory as pnml_exporter

# convert the obtained process tree into a petri net
filt_net, filt_initial_marking, filt_final_marking = pt_converter.apply(filt_tree, variant=pt_converter.TO_PETRI_NET)

#save the obtained petri net for further use
pnml_path = "filtered_petri.pnml"
pnml_exporter.apply(filt_net, filt_initial_marking, pnml_path, final_marking=filt_final_marking)

## e)

> Consider the process model for the patients who were prescript the special medication. What do you observe? How is this behavior captured by the complete model in d)?

Let's have a look at the traces that contain this activity.

In [ ]:
special_medication_traces = renamed_traces[renamed_traces.apply(lambda t: 'Prescripe Special Medication' in t)]

special_medication_traces.sample(10).values

In [ ]:
special_medication_traces[special_medication_traces.apply(lambda t: 'Treatment B' in t)]

Through further inspection we can assume that the special medication is only prescribed together with the A treatments, therefore it was damaged by the actions taken in d), as previously highlighted.

So we'll generate the model just for the patients that were prescribed special medication to.

In [ ]:
df_special = df_renaming[df_renaming['Patient'].isin(special_medication_traces.index)]

df_special

In [ ]:
log_special = log_converter.apply(df_special, parameters=param_keys)

dfg_special = dfg_discovery.apply(log_special, parameters=param_keys)

dfg_special = dfg_filtering.apply(dfg_special, parameters=param_keys)

dfg_viz = dfg_visualization.apply(dfg_special, log=log_special, parameters=param_keys)

dfg_viz.render(os.path.join(figures_dir, 'q1_e_special'),
               format='pdf',
               view=True)

In [ ]:
tree_special = inductive_miner.apply_tree_dfg(dfg_special, parameters=param_keys)

tree_graph = pt_visualizer.apply(tree_special)

# fix operations size
tree_graph.body = list(map(lambda r:r.replace('width=0.6', 'width=1'),
                           tree_graph.body))

tree_graph.render(os.path.join(figures_dir, 'q1_e_tree_special'),
                  format='pdf',
                  view=True)

It is very similar, as previously stated. One can say that the previous models failed to capture the non-local influence of the `Prescripe Special Medication` in the choice of the treatment, which it is possible to assume that must be one of the A treatments.

## f)

> Apply additional miners to the log and compare the results. Which model is the best model?

#### Alpha Miner

In [ ]:
from pm4py.algo.discovery.alpha import factory as alpha_miner
from pm4py.objects.dfg.filtering.dfg_filtering import DEFAULT_NOISE_THRESH_DF

param_keys['noiseThreshold'] = DEFAULT_NOISE_THRESH_DF
filt_dfg = dfg_filtering.apply(dfg, parameters=param_keys)

net, initial_marking, final_marking = alpha_miner.apply_dfg(filt_dfg, parameters=param_keys)

net_graph = pn_visualizer.apply(net,
                                initial_marking=initial_marking,
                                final_marking=final_marking)

# fix place size
import numpy as np
body = np.array(net_graph.body)
body[body ==  '\tnode [fixedsize=true shape=circle width=0.75]'] = '\tnode [fixedsize=true shape=circle width=1]'
net_graph.body = body

net_graph.render(os.path.join(figures_dir, 'q1_f_alpha_miner'),
                 format='pdf',
                 view=True)

#### Heuristics Miner

In [ ]:
from pm4py.algo.discovery.heuristics import factory as heuristics_miner

net, initial_marking, final_marking = heuristics_miner.apply_dfg(filt_dfg, parameters=param_keys)

net_graph = pn_visualizer.apply(net,
                                initial_marking=initial_marking,
                                final_marking=final_marking)

# fix place size
import numpy as np
body = np.array(net_graph.body)
body[body ==  '\tnode [fixedsize=true shape=circle width=0.75]'] = '\tnode [fixedsize=true shape=circle width=1]'
net_graph.body = body

net_graph.render(os.path.join(figures_dir, 'q1_f_heuristics_miner'),
                 format='pdf',
                 view=True)

The Alpha miner fails completely to most of the structures beyond the initial activities. The Heuristics miner presents a good job up until the treatments start, showing precisely the control calls and its relation with the `Inform Authority*` activities. Both fail to understand non-local influence of the `Prescripe Special Medication` and the sequencing of the A treatments and checks. Therefore, one can say the most precise model is still the one generated by the IM, even though the one generated by the HM is more readable.

# Social Network Analysis

> Discover the organizational perspective of the process. For each of the following networks, try to find a clear organizational structure and discuss the structure obtained. If no clear structure is to be found, explain why this is the case.

---

# a)
> Handover-of-Work Social Network

In [ ]:
# define the path where the resulting networks will be stored
RESULT_PATH = figures_dir

In [ ]:
from pm4py.algo.enhancement.sna import factory as sna_factory
from pm4py.visualization.sna import factory as sna_vis_factory

#discover the network
handover_net = sna_factory.apply(event_log, parameters=param_keys, variant="handover")

#create a visualization of the network
handover_viz = sna_vis_factory.apply(handover_net, variant="pyvis")

#save the visualization of the discovered network
sna_vis_factory.save(handover_viz, RESULT_PATH+"handover_net.html", variant="pyvis")


As the pyvis interactive visualization cannot be embedded, the discovered network is shown below in the form of an image. The real network can be found in the figures folder.

![](./figures/q2_handover.png)

A handover-of-work network connects two resources when an activity performed by one resource directly follows the activity performed by another resource. The more frequent this relationship is in the log, the higher is the weight of an edge. In the graphic above the handover-of-work network that was discovered from the given log can be seen.

We see that the network consists of four strongly connected main components and a single resource that has only one connection. Further we see that the resources in each of the strongly connected components all have a common first letter. The outer three components contain resources with first letters B, C and D. The inner component consists of resources with the first letter A. The nodes in the three outer components are connected only to themselves and the inner A component, but not to the other outer components. Most of the nodes in the A component have connections to all other components and the nodes in the A component have overall the most connections to other nodes. There is a single resource that does not belong to any of the components and has only one incoming edge.

From an orgainzational perspective the observed network might be explained in the following way: The outer components B, C, D all represent certain distinct departments. All of the resources in such a department do similar work and are working together on a certain part of the process. The central A component however consist of resources that are delegating and inspecting work. They pass process instances from one department to another. Since the singular resource has only incoming edges, it must be concerned with the end of the process. It might be a kind of supervisor role that makes a final assertion of a process before it ends. However this node is not involved in most cases, as its connection is rather weak.

---

# b)
> Subcontracting Social Network

In [ ]:
#discover the network
subcontracting_net = sna_factory.apply(event_log, parameters=param_keys, variant="subcontracting")

#create a visualization of the network
subcontracting_viz = sna_vis_factory.apply(subcontracting_net, variant="pyvis")

#save the visualization of the discovered network
sna_vis_factory.save(subcontracting_viz, RESULT_PATH+"subcontracting_net.html", variant="pyvis")

As the pyvis interactive visualization cannot be embedded, the discovered network is shown below in the form of an image. The real network can be found in the figure folder.

![](./figures/q2_subcontracting.png)

A subcontracting social network visualizes what resources are involved in the work of another resource.

The obtained network that can be seen above has a kind of forest structure. We again see distinct connected components that show the same A, B, C, D division as in the handover-of-work network discussed above. There is one component for each of these, except for the A component, which now consists of two not connected components. As a common structure we can observe a kind of star pattern for many nodes, meaning that one resource is involved in the work of many other resources. We can see that one of the A components has a very strong relation between two of the resources.

From an organizational perspective we can again see that the work of one department never involves the work of another department. Subcontracting only happens within the A, B, C, D departments. The star structures suggest that there are supervisor roles, resources that often subcontract work to other resources which do not subcontract any work. The strong connection between the _Alexander_ and _Ava_ resources suggests that Alexander is in a leadership position and Ava might be her secretary, as most of the work is subcontracted.

---

# c)
> Working-Together Social Network

In [ ]:
#discover the network
working_together_net = sna_factory.apply(event_log, parameters=param_keys, variant="working_together")

#create a visualization of the network
working_together_viz = sna_vis_factory.apply(working_together_net, variant="pyvis")

#save the visualization of the discovered network
sna_vis_factory.save(working_together_viz, RESULT_PATH+"working_together_net.html", variant="pyvis")

As the pyvis interactive visualization cannot be embedded, the discovered network is shown below in the form of an image. The real network can be found in the figures folder.

![](./figures/q2_working_together.png)

The working-together network depicts which resources are often working together on the same case, so in which fraction of cases they were both involved.

We see a similar structure as in the hondover-of-work network, which is expected. We again have four main components, with three outer ones and an inner one, following the recurring A, B, C, D pattern. We see that the connections in this network are even stronger than in the handover of work network. All of the components are almost fully connected. The inner A component is most strongly connected. Again the outer components have no connections to the other outer connections, but are all connected to the inner component. The singular resource is again present and is only connected to resources in the A component.

From an organizational point of view, we see that the resources in the A component are the most important ones. They are involved in almost all cases and are almost always working together. Again it seems that they delegate work to the outer components. The almost full connection of the components suggests that when a department is involved in a case at all, usually the whole department is involved and not only certain resources. From the subcontracting network we have seen that in the outer components, much of the work is subcontracted, which explains the involvement of many different resources of each component we can see in the working-together network.

--- 

# d)
> Joint-Activity Network

In [ ]:
#discover the network
jointactivities_net = sna_factory.apply(event_log, parameters=param_keys, variant="jointactivities")

#create a visualization of the network
jointactivities_viz = sna_vis_factory.apply(jointactivities_net, variant="pyvis")

#save the visualization of the discovered network
sna_vis_factory.save(jointactivities_viz, RESULT_PATH+"jointactivities_net.html", variant="pyvis")

As the pyvis interactive visualization cannot be embedded, the discovered network is shown below in the form of an image. The real network can be found in the figures folder.

![](./figures/q2_jointactivities.png)

The jointactivities network depicts how similar the types of activities are that the resources perform. Meaning that two resources have a strong connection if they always perform the same e.g. three activities but never any other.

The structure of the obtained network is very different from all networks we have considered up to this point. It consists of six strongly connected components. It seems that all of the components are fully connected, meaning that all resources in a component perform very similar activities. We see that the destinction between components is no longer based on the first letter of the resource name, as we have seen before. The components are now mixtures of these resources. This is not true for one of the components, which only contains A resources.

For the organizational structure this again suggests the supervisor role for the A resources, there are certain activities that are only performed by these resources. Other activities however seem to occur in all of the B, C, D departments. Combining these findings with the findings from the other networks we cann assume that the process is structured in such a way that the A department is a supervision department that performs certain specialized taks. The other departments B, C, D, however perform similar activities (as the jointactivities network suggest) but they never mix. This suggests that a case is delegate by the A resources to one of the departments and this department then fully handles the case. It is in a way not a delegation of cases based on certain types of cases, but just a balancing of cases over all departments.

In the following we are taking a closer look at the relation between groups and activities.

In [ ]:
df_log['Resource']

In [ ]:
first_person_group = df_log['Resource'].str[0]
second_person_group = df_log['Resource'].str.split(',').apply(lambda a: a[-1]).str[0]

(first_person_group != second_person_group).sum()

This means that when two resources share the activity, it is contained in one of the groups of strongly connected resources.

In [ ]:
df_log['Component'] = first_person_group

df_log['Component'].value_counts()

In [ ]:
for c in df_log['Component'].unique():
    print('==== Component ', c)
    print(df_log[df_log['Component'] == c]['Activity'].value_counts())

We verify that, indeed, the components represent groups of resources that act in different activities. We can estimate that groups B, C and D are responsible for the treatment of confirmed cases; group I is responsible for the control call only; H is the helicopter, only used in emergencies; and group A deals with the remaining activities related to the trial of the patients.

### What are the components of the jointacitivites

Right-most component:

In [ ]:
df_log[df_log['Resource'] == 'Daniela']['Activity'].unique()

Most dense ball:

In [ ]:
df_log[df_log['Resource'] == 'Benedikt,Bella']['Activity'].unique()

In [ ]:
df_log[df_log['Resource'] == 'Babatunde,Bruno']['Activity'].unique()

In [ ]:
df_log[df_log['Resource'] == 'Celina,Cedrik']['Activity'].unique()

Line segment:

In [ ]:
df_log[df_log['Resource'] == 'Adrian']['Activity'].unique()

Octogon:

In [ ]:
df_log[df_log['Resource'] == 'Adrian,Amalia']['Activity'].unique()

Square:

In [ ]:
df_log[df_log['Resource'] == 'Anna']['Activity'].unique()

Left-most component:

In [ ]:
df_log[df_log['Resource'] == 'Diana']['Activity'].unique()

# Performance Analysis

> Which parts of the process have the biggest influence on the total case duration?

## Data loading

Import the log.

In [ ]:
from pm4py.objects.log.importer.xes import factory as xes_import_factory
from pm4py.objects.petri.importer import factory as pnml_importer

#import the renamed event log from Q1
event_log = xes_import_factory.apply("processed_log.xes")

#import the filtered petri net obtained in Q1
pnml_path = "filtered_petri.pnml"
net, initial_marking, final_marking = pnml_importer.apply(pnml_path)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# set default plot size
plt.rcParams['figure.figsize'] = (15, 10)
plt.rcParams['figure.figsize'] = (15, 10)

---

# a)
> Provide and briefly describe results of your performance analysis. Remember to also consider your current results which may give you a good entry point for a deeper analysis.

### Activities duration

We start our exploration of the process performance by evaluating the performance of single activities. At first we visualize the activities duration. The results can be found below.

In [ ]:
ax = sns.boxplot(data=df_log, y='Activity', x='@@duration', orient='h')
ax.set_title("Boxplots of the Activity Service Times", size=25)
ax.set_ylabel("Activity", size=20)
ax.set_xlabel("Service Time in seconds", size=20)
plt.show()

What we can observe here is that the _Prescripe Special Medication_ activity has by far the largest average duration. Also, _Test III_ has the second largest average duration.

Administrative tasks like registration activities also show a long duration, while examination decisions and treatment checks have the shortest duration.

All of the standard deviations are small considering the overall duration of the activities, this means that there is not much variation in the time that is required for a specific activity in between cases. We also notice that there is no presence of anomalies in the distributions, all of them seem very close to a normal distribution.

In general we see that the activities related to treatments have long durations, we can take a closer look.

In [ ]:
ax = sns.boxplot(data=df_log[df_log['Activity'].str.contains('Treatment')], y='Activity', x='@@duration', orient='h')
ax.set_title("Service Time Boxplots of Treatment Related Activities", size=25)
ax.set_ylabel("Activity", size=20)
ax.set_xlabel("Service Time in seconds", size=20)
plt.show()

We see that _Treatment A2_ has the largest average duration, but knowing that this activities are performed many times in a single process, it is useful to evaluate their summed duration.

In [ ]:
df_activities_duration = (df_log.groupby(['Patient', 'Activity'])['@@duration'].sum()
                                .reset_index().rename({'@@duration': 'total_@@duration'}, axis='columns'))
df_treatment_durations = df_activities_duration[df_activities_duration['Activity'].str.contains('Treatment')]

ax = sns.boxplot(data=df_treatment_durations, y='Activity', x='total_@@duration', orient='h')
ax.set_title("Service Time Spent on Treatment Related Activities per Case", size=25)
ax.set_ylabel("Activity", size=20)
ax.set_xlabel("Total Service Time of the Activity in Seconds", size=20)
plt.show()

So we see that even though _Treatment A2_ has the biggest average duration, it is not the one with the largest total duration (taking on account all the times it was executed) in a process, this being _Treatment A1_.

We can also notice that when we are dealing with the total duration of the activities, the values become bigger by one order of magnitude, which makes us interested in the total duration of the remaining activities.

In [ ]:
df_ = df_activities_duration[~df_activities_duration['Activity'].str.contains('Treatment')]

ax = sns.boxplot(data=df_, y='Activity', x='total_@@duration', orient='h')
ax.set_title("Service Time Spent on Other Activities per Case", size=25)
ax.set_ylabel("Activity", size=20)
ax.set_xlabel("Total Service Time of the Activity in Seconds", size=20)
plt.show()

This way we can see that the _Control Call_ activity is actually more time demanding than we have estimated by the previous analysis.

In [ ]:
total_process_duration = df_log.groupby('Patient')['@@duration'].sum()

In [ ]:
total_process_duration = df_log.groupby('Patient')['@@duration'].sum()
df_activities_duration['process_total_duration'] = df_activities_duration['Patient'].map(total_process_duration)

ax = sns.boxplot(data=df_activities_duration, y='Activity', x='process_total_duration')
ax.set_title("Total Case Service Time Boxplots", size=25)
ax.set_ylabel("Activity", size=20)
ax.set_xlabel("Total Service Time in Seconds of the Case", size=20)
plt.show()

Comparing the activities with the total process duration in which they are present, we see that the Treatments A1, A2 and A3 correlate to much longer processes.

---

# b)
> Discuss insights obtained from you analysis, for example identify bottlenecks, and discuss their impact.

### Process length

We could notice in the previous graph that the processes can be divided in two groups regarding their length, let's take a deeper look into that.

In [ ]:
ax = sns.distplot(total_process_duration)
ax.set_title("Distribution of Total Case Service Time", size=25)
ax.set_xlabel("Total Service Time of Cases in Seconds", size=20)
plt.show()

We notice roughly two groups here, separated by the 40000 seconds duration mark.

Let's investigate further if this is also reflected in the lead time of the processes.

In [ ]:
lead_time = df_log.groupby('Patient')['Timestamp'].max() - \
            df_log.groupby('Patient')['start_timestamp'].min()

ax = sns.distplot(lead_time.apply(lambda t: t.total_seconds()), bins=20)
ax.set_title("Distribution of Total Case Lead Time", size=25)
ax.set_xlabel("Total Lead Time of Cases in Seconds", size=20)
plt.show()

Again we see that the lead time has two groupings, split around 370000 seconds.

We can search for the causes of this difference by seeing it as a classification problem being the features the presence of the activities. This way, the most influential features as the main causes for the lead time.

In [ ]:
from pm4py.objects.log.util import get_log_representation, get_class_representation

data, feature_names = get_log_representation.get_representation(
    log=event_log,
    str_tr_attr=[],
    str_ev_attr=['Activity'],
    num_tr_attr=[],
    num_ev_attr=[]
)
target, classes = get_class_representation.get_class_representation_by_trace_duration(event_log, target_trace_duration=370000, timestamp_key='Timestamp')

from sklearn.tree import DecisionTreeClassifier

# limit DT depth to get a simpler representation, with the main approaches
clf = DecisionTreeClassifier(min_samples_leaf=5)
# clf = DecisionTreeClassifier(min_samples_leaf=15)
clf = clf.fit(data, target)

from pm4py.visualization.decisiontree import factory as dt_visualizer

dt_vis = dt_visualizer.apply(clf, feature_names, classes)

dt_vis.render(os.path.join(figures_dir, 'q3_decision_tree'),
              #format='pdf',
              view=True)

We can see that the major cause for the long lead time of the processes is the happening of `Inform Authority Send Form`, which means that the patient will be forwarded to treatment, therefore taking longer time under observation.

In [ ]:
from pm4py.visualization.petrinet import factory as pn_visualizer
from pm4py.algo.discovery.inductive import factory as inductive_miner
from pm4py.util import constants

# map dataset columns to PM4Py keys
param_keys={constants.PARAMETER_CONSTANT_CASEID_KEY: 'Patient',
            constants.PARAMETER_CONSTANT_RESOURCE_KEY: 'Resource', 
            constants.PARAMETER_CONSTANT_ACTIVITY_KEY: 'Activity',
            constants.PARAMETER_CONSTANT_TIMESTAMP_KEY: 'Timestamp',
            constants.PARAMETER_CONSTANT_START_TIMESTAMP_KEY: 'start_timestamp'}

#annotate the mined petri net with performance measures
perf_net_vis = pn_visualizer.apply(net, initial_marking, final_marking=final_marking,
                                   variant=pn_visualizer.PERFORMANCE_DECORATION,
                                   log=event_log, parameters=param_keys)

# fix place size
import numpy as np
body = np.array(perf_net_vis.body)
body[body ==  '\tnode [fixedsize=true shape=circle width=0.75]'] = '\tnode [fixedsize=true shape=circle width=1]'
perf_net_vis.body = body

perf_net_vis.render(os.path.join(figures_dir, 'q3_perf_petrinet'),
                 #format='pdf',
                 view=True)

We have annotated the petri net that was obtained in Q1 after preprocessing the log with a performance metric. The metric used is the mean time between two events.

### Components

We can also visualize if the components (as discussed in Q2) have any impact in the performance of the processes.

In [ ]:
df_log['Component'] = df_log['Resource'].str[0]

df_components = df_log.groupby(['Component', 'Patient'])['@@duration'].sum().reset_index()

ax = sns.boxplot(data=df_components, x='Component', y='@@duration')
ax.set_title("Boxplots of Components against Total Service Time", size=25)
ax.set_xlabel("Component Type", size=20)
ax.set_ylabel("Total Service Time in Seconds", size=20)
plt.show()

We see that the component C is presenting longer processes duration for the activities than their counterparts B and D, which perform similar activities.

In [ ]:
df_treatment = df_log[(df_log['Component'].isin(['B', 'C', 'D'])) & \
                      (df_log['Activity'].str.contains('Treatment'))]

ax = sns.boxplot(data=df_treatment, x='Activity', y='@@duration', hue='Component')
ax.set_title("Boxplots of Treatment Service Times for Different Components", size=25)
ax.set_xlabel("Activity", size=20)
ax.set_ylabel("Service Time in Seconds", size=20)
plt.show()

We see no difference in the performance of the activities comparing the three components that deal with the treatments. Therefore, we can estimate that the difference is in the amount of activities performed in the processes related to component C.

In [ ]:
activities_per_component = df_log.groupby('Component')['Activity'].count()
patients_per_component = df_log.groupby('Component')['Patient'].nunique()

activities_per_component / patients_per_component

We confirm our idea that the component C performs more tasks for each related process, having an average of 31 activities per process (in which it actuates).

### Age

We can also inspect if the age of the patients has any influence on the performance.

In [ ]:
g = sns.jointplot(data=df_log, x='@@duration', y='case:Age')
#g.ax_joint.set_title("Scatterplot of the Activity Service Time Against the age", size=25)
g.ax_joint.set_xlabel("Activity Service Time in Seconds", size=20)
g.ax_joint.set_ylabel("Patient Age in Years", size=20)
plt.show()

In [ ]:
df_ = pd.DataFrame(total_process_duration).rename({'@@duration':'total_duration'}, axis='columns')
df_['case:Age'] = df_log.groupby('Patient')['case:Age'].first()

g = sns.jointplot(data=df_, x='total_duration', y='case:Age')
g.ax_joint.set_xlabel("Total Case Service Time in Seconds", size=20)
g.ax_joint.set_ylabel("Patient Age in Years", size=20)
plt.show()

So we see that neither the duration of the individual activities nor the total duration of the process are related to the age of the patients.

### Insurance

In [ ]:
for ins in df_log['case:Insurance'].unique():
    ax = sns.distplot(df_log[df_log['case:Insurance'] == ins]['@@duration'], label=ins)
    ax.set_title("Distribution of Activity Service Times for Different Insurance Types", size=25)
    ax.set_xlabel("Activity Service Time", size=20)

plt.legend()
plt.show()

In [ ]:
df_['case:Insurance'] = df_log.groupby('Patient')['case:Insurance'].first()

for ins in df_['case:Insurance'].unique():
    ax = sns.distplot(df_[df_['case:Insurance'] == ins]['total_duration'], label=ins)
    ax.set_title("Distribution of Total Service Times for Different Insurance Types", size=25)
    ax.set_xlabel("Total Service Time", size=20)
plt.legend()
plt.show()

Again, no apparent influence.

In [ ]:
case_duration = (df_log.groupby('Patient')['Timestamp'].max() - 
                    df_log.groupby('Patient')['start_timestamp'].min())
case_duration = lead_time.apply(lambda x: x.total_seconds())
case_duration.name = 'case_duration'

As we have seen there is a group of cases that take substantially longer than the majority of cases we will now focus on this group for further analysis

In [ ]:
new_cc_name = 'Control Call (+)'
split_on_activity = 'Test III'

# get `Test III` moment
df_log_test = df_log[df_log['Activity'] == split_on_activity]
split_timestamp = df_log_test.groupby('Patient')['Timestamp'].first()

# map timestamp to whole patient trace
df_renaming = df_log.copy()
df_renaming[split_on_activity + ' Timestamp'] = df_log['Patient'].map(split_timestamp)

# renames `Control Call` activities that happen after `Test III`
new_cc = df_renaming['Activity'].str.replace('Control Call', new_cc_name)
df_renaming['Activity'] = new_cc.where(
    df_renaming['Timestamp'] > df_renaming[split_on_activity + ' Timestamp'],
    df_renaming['Activity']
)

In [ ]:

long_cases = df_renaming.join(case_duration ,on='Patient')
long_cases = long_cases[long_cases['case_duration']>300000]


We can now have a look at the dfg visualisations of the long cases both from a performance and a frequency perspective:

In [ ]:
long_cases_log = log_converter.apply(long_cases, parameters=param_keys)

dfg = dfg_discovery.apply(long_cases_log, variant="performance", parameters=param_keys)
gviz = dfg_visualization.apply(dfg, log=long_cases_log, variant="performance")
dfg_visualization.view(gviz)

In [ ]:
dfg = dfg_discovery.apply(long_cases_log, variant="frequency", parameters=param_keys)
gviz = dfg_visualization.apply(dfg, log=long_cases_log, variant="frequency")
dfg_visualization.view(gviz)

When we look at the frequency graph we can see that the control call activities are both very often repeated and have a high mean lead time of 7 or 8 hours. They are a big factor in the long running cases.

In [ ]:
print(long_cases['Activity'].value_counts()[:2])
print(df_renaming['Activity'].value_counts()[:2])

A brief comparison of the number of control call activities shows that all control calls after a positive test are still included. This is normal because patients that receive a positive test results will naturally have longer case durations. What is really interesting is that around 72% of the control calls before the test are still included in the long cases. This means that some of the long running cases are caused by patients waiting a long time for a test (and receiving calls while they do).

# Decision Points

> Investigate how patients are referred for further treatment by means of a decision tree. Describe the factors that you observe.

## Data loading

Import the original log and modify column names and datatypes for the following analysis.

In [ ]:
# load csv from disk
df_log_Q4 = pd.read_csv("log.csv")

# convert timestamp columns to datetime friendly format
df_log_Q4['Timestamp'] = pd.to_datetime(df_log_Q4['Timestamp'])
df_log_Q4['start_timestamp'] = pd.to_datetime(df_log_Q4['start_timestamp'])

#rename some column for better algorithm compatibillity
df_log_Q4 = df_log_Q4.rename(columns={"Age": "case:Age", "Insurance": "case:Insurance", "PatientName": "case:PatientName", "Timestamp": "time:timestamp"})


---

# a)
> Create a decision tree of reasonable complexity using the available attributes in the log. 

In this section we are interested in the kind of treatment that a patient will undergo based on certain properties.  For this we want to create decision trees based on case attributes that can give us deeper insights into which patients recieve which treatment.

To do so we are interested in which cases contain certain treatment related events. PM4Py offers a decision tree module, that creates a decision tree predicting the end event of a case from the case properties. Since we know that eventually all patients will be discharged from our previous analysis, most of the cases will have a discharge end event. From this we cannot infer which treatment was performed before the discharge. Because of this, before creating the decision tree, we will cut off all traces when a certain event occurs. This event is one of a set of events that describe the treatment of the patient. The events included in this set can be seen below. They are either related to a certain kind of treatment, or a discharge. The discharge events are included for cases that do not recieve any treatment.

In [ ]:
NEW_END_ACTIVITIES = ["Treatment A1", "Treatment A2", "Treatment B", "Discharge", "Discharge Test", "Discharge Init Exam"]

In the following step the original event log is filtered in such a way, that all events that occurred after one of the above events in a case will be discarded.

In [ ]:
#remove all events after a final decision was made (slow)
treatment_df = pd.DataFrame(index=np.arange(0, len(df_log_Q4)), columns=["case:concept:name", "concept:name", "org:resource", "case:PatientName", "case:Age", "case:Insurance", "start_timestamp", "Timestamp", "@@duration"])

current_id = -1
keep = True
count = 0

# go through the sorted events on a case basis
# if an event from the new end activities is found, discard all further events from that case
for row in df_log_Q4.itertuples():
    if current_id != row[1]:
        current_id = row[1]
        keep = True

    if keep:
        if row[2] in NEW_END_ACTIVITIES:
            keep = False
            
        treatment_df.loc[count] = row[1:]
        count += 1         

# drop nil rows that will occur because after filtering there are fewer events
treatment_df = treatment_df.dropna()


In [ ]:
treatment_df.head(13)

We can see that case 1 now ends with Treatment B instead of an discharge event. In the next step we will convert the log into an PM4Py event log as seen before.

In [ ]:
#convert to event log
# map dataset columns to PM4Py keys
param_keys_Q4 = {constants.PARAMETER_CONSTANT_CASEID_KEY: 'case:concept:name',
            constants.PARAMETER_CONSTANT_RESOURCE_KEY: 'org:resource', 
            constants.PARAMETER_CONSTANT_ACTIVITY_KEY: 'concept:name',
            constants.PARAMETER_CONSTANT_TIMESTAMP_KEY: 'time:timestamp',
            constants.PARAMETER_CONSTANT_START_TIMESTAMP_KEY: 'start_timestamp'}

treatment_log = log_converter.apply(treatment_df, parameters=param_keys_Q4)

Since not all cases contain one of the new end activities, there are still traces in the event log that do not end with one of the specified activities. In order to keep just the cases that end on one of the specified events we use the end activitiy filter provided by PM4Py.

In [ ]:
from pm4py.algo.filtering.log.end_activities import end_activities_filter

treatment_log = end_activities_filter.apply(treatment_log, NEW_END_ACTIVITIES)

After preprocessing the log we can now create a decision tree from it. In the first step of this, we use PM4Py to create the data, targets and classes that will be passed to the sklearn decision tree algorithm. In this step we need to specify which properties of the event log should be used for the decision tree creation. We can speecifiy trace based and event based attributes.

For our first iteration we included all of the sensible log attributes (for example patient name was removed as it provides no information).

In [ ]:
from pm4py.objects.log.util import get_log_representation
from pm4py.objects.log.util import get_class_representation

# preprocess the log for decision tree mining
str_trace_attributes = ["Insurance"]
str_event_attributes = ["org:resource", "concept:name"]
num_trace_attributes = ["Age"]
num_event_attributes = ["@@duration"]

data, feature_names = get_log_representation.get_representation(treatment_log, str_trace_attributes, str_event_attributes,
                                                              num_trace_attributes, num_event_attributes)

target, classes = get_class_representation.get_class_representation_by_str_ev_attr_value_value(treatment_log, "concept:name")

In [ ]:
from sklearn import tree

#calculate the decision tree

# 0 treatmentB, 1 treatment A1, 2 discharge test, 3 discharge init, 4 treatment a2
classifier = tree.DecisionTreeClassifier()
classifier.fit(data, target)

In [ ]:
from pm4py.visualization.decisiontree import factory as dt_vis_factory
#visualize the obtained decision tree

decision_tree_vis = dt_vis_factory.apply(classifier, feature_names, classes)

decision_tree_vis.render(os.path.join(figures_dir, 'q4_tree_all'),
                 format='pdf',
                 view=True)

The obtained tree is able to perfectly classify all cases, which is unexpected. A closer investigation shows that the tree simply uses the target events themselves to classify the cases, which of course is not wanted. As some events are always performed by the same resources, the same holds for the resource attribute. The two attributes are therefore not suited to be included in a sensible decision tree for this taks.

Furthermore we see that the tree once splits using the duration of an event. Since the tree holds no information about which event is related to this duration, we cannot derive meaningful results from this split. The attribute is therefore removed as well. With the remaining age and insurance attributes, we create a new decision tree in the following.

In [ ]:
# preprocess the log for decision tree mining
str_trace_attributes = ["Insurance"]
str_event_attributes = []
num_trace_attributes = ["Age"]
num_event_attributes = []

data, feature_names = get_log_representation.get_representation(treatment_log, str_trace_attributes, str_event_attributes,
                                                              num_trace_attributes, num_event_attributes)

target, classes = get_class_representation.get_class_representation_by_str_ev_attr_value_value(treatment_log, "concept:name")


Since the _Discharge Init Exam_ event is disproportionally more frequent than all other classes, we need to introduce some normalization in order to obtain a sensible tree. (Else all nodes would have the same label for a small tree) Additionally, since the tree is not able to perfectly classify all cases anymore, we have to limit the depth and maximum number of child nodes in order to obtain readable results.

In [ ]:
#calculate the decision tree

# 0 treatmentB, 1 treatment A1, 2 discharge test, 3 discharge init, 4 treatment a2
classifier = tree.DecisionTreeClassifier(max_depth=7,max_leaf_nodes=8,class_weight={0: 0.6, 1: 1, 2: 1, 3: 0.5, 4: 2})
classifier.fit(data, target)

In [ ]:
#visualize the obtained decision tree

decision_tree_vis = dt_vis_factory.apply(classifier, feature_names, classes)

decision_tree_vis.render(os.path.join(figures_dir, 'q4_tree_min'),
                 format='pdf',
                 view=True)

The new decision tree with an reduced attribute set is much more interpretable. For example we see that patients older than 63.5 years that have a state insurance are more often discharged after the initial exam than ones with a private insurance. This group recieves treatment B more frequently.

Furthermore we can observe, that really young patients (<=15.5) are more often discharged after having been tested, while older patients between 15.5 and 39.5 years are more often directly discharged after thee initial exam.

In general we can observe that privately insured patients recieve any kind of treatment more frequently and are less often discharged without treatment.


---

# b)
> Since it is likely that the resources at the treatment facilities are limited, implement a function that assigns a(n) (estimate) of the number of patients at each facility to each event. To this end, you have to decide which event occurs at which facility based on your analysis in question 2. Create a decision tree of reasonable complexity using this derived attribute.

Based on our analysis in Q2, we know that the respurces are heavily related to the facilities. More concrete the first letter of the name of the resource corresponds to the facility of the resource. We will therefore use this property for the distinction between the different facilities.

For this we first create a new property called facillity that is derived from the first letter of the resource of the event. Based on this we add a new column that documents the move of a patient between two facilities.

In [ ]:
def create_fac(row):
    return row["org:resource"][0]

def create_fac_move(row):
    patient = row["case:concept:name"]
    last_patient = row["last_patient"]
    next_patient = row["next_patient"]
    
    facility = row["facility"]
    last_facility = row["last_facility"]
    
    if patient == last_patient:
        if next_patient == patient:
            if last_facility is not None:
                return str(last_facility)+"->"+str(facility)
            else:
                return "start->"+str(facility)
        else:
            return str(facility)+"->end"
    else:
        return "start->"+str(facility)
    

df_fac = treatment_df.copy()

#obtain the facility from the resource name for all events
df_fac["facility"] = df_fac.apply(lambda row: create_fac(row), axis=1)

#Find out which facilities exist
pd.unique(df_fac["facility"])

In [ ]:
#shift the facility and patient column down, so each row knows what the last facility and patient was
df_fac["last_facility"] = df_fac["facility"].shift(1)
df_fac["last_patient"] = df_fac["case:concept:name"].shift(1)

#shift the patient row up so we know what the next patient is (important for last event)
df_fac["next_patient"] = df_fac["case:concept:name"].shift(-1)

#create a move in the form last_facility->current_facility for each event
df_fac["fac_move"] = df_fac.apply (lambda row: create_fac_move(row), axis=1)

df_fac = df_fac.drop(["facility", "last_facility", "last_patient", "next_patient"], 1)
df_fac.head(24)

As can be seen above, each event now contains the information about the facility move that has happened. We assume that the move between facilities happens at the start time of the current event. We can now use this move record to calculate for each event, how many patients currently are assigned to a certain facility.


In [ ]:
FACILITIES = ["A", "B", "C", "D", "I", "H"]
results = np.zeros(len(df_fac))

def get_fac_count(facility_index, row, index):
    facility = FACILITIES[facility_index]
    facs = row[1].split("->")

    old_fac = facs[0]
    current_fac = facs[1]

    old_count = 0
    if index > 0:
        old_count = results[index-1]
    else:
        #set fitting initial values, obtained from a prior run of the algorithm
        if facility == "A":
            old_count = 10
        elif facility == "I":
            old_count = 20
        elif facility in ["A", "B", "C", "D"]:
            old_count = 1
        
    new_count = old_count

    #increase / decrease count based on facility move
    if old_fac != current_fac:
        if current_fac == facility:
            new_count = old_count + 1
        elif old_fac == facility:
            if old_count > 0:
                new_count = old_count - 1
            
    return new_count

#sort the dataframe by start timestamp
df_fac = df_fac.sort_values("start_timestamp").reset_index(drop=True)

df_count = df_fac[["fac_move"]].copy()

# assign patient counts to all events for all facilities
for facility_index in range(0, len(FACILITIES)):
    facility = FACILITIES[facility_index]
    results = np.zeros(len(df_fac))
    index = 0
    
    for row in df_count.itertuples():
        results[index] = get_fac_count(facility_index, row, index)
        index = index + 1

    df_count[facility] = results
    
# join with the original dataframe to keep all value columns  
df_fac_count = df_fac.join(df_count[FACILITIES])


Below we can see the resulting dataframe with the values for the facilities.

In [ ]:
df_fac_count.head(15)

As can be seen in the dataframe above, we have assigned an initial patient count to all of the facilities. This was done because the given dataset is presumably not complete and the facilities have also operated in the time that is not captured by the dataset. The initial values were chosen on the metrics that can be seen below. A fitting initial value for the facilities was chosen based on the mean value of patients from a previous run of the algorithm and a visual interpretation of the patient counts.

Below, the patient counts of two of the facilities are plotted (A blue, I orange). We can see that there is some kind of weekly pattern and that the patient count in facility A steadily rises up until the end.

In [ ]:
from matplotlib import pyplot as plt
pd.plotting.register_matplotlib_converters()

In [ ]:
plt.figure(figsize=(20,5))
plt.plot(df_fac_count["start_timestamp"], df_fac_count["A"], label="Facility A")
plt.plot(df_fac_count["start_timestamp"], df_fac_count["I"], label="Facility I")
plt.xlabel("Start Timestamp")
plt.ylabel("Patient Count")
plt.legend(loc="upper left")

plt.show()


In [ ]:
# check what initial value to assing to the patient counts
df_fac_count.mean()

In the next steps, the obtained dataframe with the counts is transformed into an event log again. As parameters for the decision tree we only choose the patient count of the facilities. Again we want to predict the kind of treatment a patient recieves.

In [ ]:
fac_count_log = log_converter.apply(df_fac_count, parameters=param_keys_Q4)

In [ ]:
# preprocess the log for decision tree mining
str_trace_attributes = []
str_event_attributes = []
num_trace_attributes = []
num_event_attributes = FACILITIES

data, feature_names = get_log_representation.get_representation(fac_count_log, str_trace_attributes, str_event_attributes,
                                                              num_trace_attributes, num_event_attributes)

target, classes = get_class_representation.get_class_representation_by_str_ev_attr_value_value(fac_count_log, "concept:name")


In [ ]:
#calculate the decision tree

# 0 treatmentB, 1 treatment A1, 2 discharge test, 3 discharge init, 4 treatment a2
classifier = tree.DecisionTreeClassifier(max_depth=8,max_leaf_nodes=8,class_weight={0: 0.6, 1: 1, 2: 1, 3: 0.5, 4: 2})
classifier.fit(data, target)

In [ ]:
#visualize the obtained decision tree

decision_tree_vis = dt_vis_factory.apply(classifier, feature_names, classes)

decision_tree_vis.render(os.path.join(figures_dir, 'q4_facility_tree'),
                 format='pdf',
                 view=True)

We obtain a decision tree that is splitting the cases based on the number of patients in each of the facilities. 

The first split is done on the A facility. We observe that when there are fewer (less than 21) patients in facility A, more treatments are prescribed and less patients are discharged at the initial exam. This could mean that the facility is somehow overloaded when there are many patients, which influences the decision making. 

On the second level we can also observe that there are more initial discharges if there are not many patients in facility C. As discovered in Q2, facility C is mainly concerned with checking treatments, so the reason here might be that when there are more people discharged, there is less need for treatment checks. On the other hand this may also indicate a potential bottleneck in C: There are not enough resources to perform checks, so patients are more frequently discharged.

We also see that the facilities B and D were never used as a splitting criterion, which might indicate that the number of patients in these facilities has no big influence on the treatment and therefore there are enough resources in these facilities.

Facility I is a good indicator for the overall number of _active_ patients that are currently in the process, as patients stay in that facility for a longer time (for the control calls). The I facality patient count is therefore also used as a splitting criterion for some nodes. We again observe that treatments are less frequent for higher patient numbers.

In a last step we calculated the number of resources in each facility to compare those numbers with the obtained patient numbers. The results can be seen below.


In [ ]:
from collections import Counter

# find out how many resources there are in every facility
resources = list(df_log_Q4["Resource"])
resources = [r for rs in (resource.split(",") for resource in resources) for r in rs]
resources = list(set(resources))
resources = [r[0] for r in resources]
Counter(resources)

We see that, altough facility A has the highest patient load, it has the lowest number of resources compared to the other "normal" facilities B, C and D. This is a further indicator that facility A might be understaffed.

We also see that there is only one resource in facility I, we have already observed this fact in Q2. This is probably the case as the control calls are somehow automated and performed by a single system, so there can be no resource bottlenecks here.